In [1]:
!pip install transformers datasets

     |████████████████████████████████| 2.3MB 14.5MB/s 
     |████████████████████████████████| 225kB 48.0MB/s 
     |████████████████████████████████| 3.3MB 52.0MB/s 
     |████████████████████████████████| 901kB 56.5MB/s 
     |████████████████████████████████| 245kB 53.5MB/s 
     |████████████████████████████████| 112kB 62.1MB/s 


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch

import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification

from datasets import load_dataset

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report as clf_report
from sklearn.metrics import (
    accuracy_score,
    recall_score,
    precision_score,
    f1_score
)

In [3]:
MODEL = "allenai/scibert_scivocab_uncased"
BATCH_SIZE = 16
MAX_LEN = 126

In [6]:
dataset = load_dataset('csv', data_files={'train': '10percent_train_tech.csv',
                                          'test': '10percent_test_tech.csv'})

Using custom data configuration default-c0a572a48648001e


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c0a572a48648001e/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [7]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [8]:
def preprocess(examples):
    encoding = tokenizer(examples["cleaned_abstract"], max_length=256, truncation=True, padding=True)
    encoding["label"] = examples["field_Tech"]
    return encoding

In [9]:
encoded_dataset = dataset.map(preprocess, batched=True)

In [10]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2)

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

In [11]:
args = TrainingArguments(
    "scibert-test",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True
)

In [14]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer
)

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.115800,0.115068


TrainOutput(global_step=9923, training_loss=0.11937187300157993, metrics={'train_runtime': 5192.1442, 'train_samples_per_second': 1.911, 'total_flos': 0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 1646936064, 'init_mem_gpu_alloc_delta': 441169408, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 29077504, 'train_mem_gpu_alloc_delta': 1384607232, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 4591635968})

In [17]:
y_pred = []
y_true = []

for idx in tqdm(range(dataset["test"].shape[0])):
  device = "cuda:0"
  model = model.to(device)
  input_ids = tokenizer(
      dataset["test"][idx]["cleaned_abstract"], return_tensors="pt",
      max_length=256, truncation=True, padding=True
      )["input_ids"]
  input_ids = input_ids.to(device)
  outputs = model(input_ids=input_ids)
  y_true.append(dataset["test"][idx]["field_Tech"])
  y_pred.append(np.argmax(outputs["logits"].cpu().detach().numpy()))

In [18]:
print(clf_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98     37981
           1       0.69      0.38      0.49      1709

    accuracy                           0.97     39690
   macro avg       0.83      0.69      0.74     39690
weighted avg       0.96      0.97      0.96     39690



In [24]:
trainer.save_model('scibert_model_trainer_tech')
trainer.args.output_dir = 'scibert_trainer_state_tech'
trainer.save_state()

In [21]:
import pickle

with open('y_pred.pkl', 'wb') as y_pred_file:
  pickle.dump(y_pred, y_pred_file)